In [ ]:
from __future__ import print_function # python2 compatibility
import asyncio
from ircclient import IRCClient
from hojong_py_util.util import parse_url

import pymysql as mysql # python3.5 mysql connector
import pymongo

#################### twitch IRC server info ####################

SERVER = 'irc.chat.twitch.tv'
CHANNEL = '#...' # set the channel you want to join
NICK = '...' # set your nickname
PASS = 'oauth:token' # you can get it from https://twitchapps.com/tmi/

#################### freenode irc server example info ####################

#'''
from random import random

SERVER = 'chat.freenode.net:6666'
CHANNEL = '#hojong'
NICK = 'hjbot_%05d' % (random() * 10000)
PASS = None
#'''

#################### mysql variables ####################
HOST = 'localhost'
USER = 'root'
PASS = 'pass'
DB = 'test_db'
TB = 'chat_tb'

conn = None
cur = None
query = 'INSERT INTO %s VALUE(%s, %s)' % (TB, nick, chat)

#################### mongodb variables ####################
HOST = 'localhost'
PORT = 27017
DB = 'test_db'
COLL = 'chat_list'

cli = None
coll = None

#################### callback ####################

def print_chat(nick, chat): # callback
    print('*** [RECEIVED] %s: %s' % (nick, chat))

#################### mysql callback ####################

def init_mysql(): # callback
    print('init_mysql')
    global conn, cur
    conn = mysql.connect(host=HOST, user=USER,password=PASS,db=DB,charset='utf8mb4',cursorclass=mysql.cursors.DictCursor)
    cur = conn.cursor()

def log_to_mysql(nick, chat): # callback
    print('log_to_mysql')
    global conn, cur
    cur.execute(query)
    conn.commit()

def close_mysql(): # callback
    print('close_mysql')
    global conn, cur
    cur.close()
    conn.close()
    
#################### mongodb callback ####################
    
def init_mongodb():
    print('init_mongodb')
    global cli, coll
    cli = pymongo.MongoClient(host=HOST, port=PORT)
    db = cli[DB]
    coll = db[COLL]

def log_to_mongodb(nick, chat): # callback
    print('log_to_mongodb')
    global coll
    coll.insert_one({'nick': nick, 'chat': chat})

def close_mongodb():
    print('close_mongodb')
    global cli
    cli.close()
    
#################### main ####################
    
def main():
    host, port = parse_url(SERVER)
    assert (PASS != 'oauth:token'), 'you must set PASS'
    print('*** [SETTING_INFO] %s %d %s %s %s' % (host, port, CHANNEL, NICK, PASS))
    
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
    # if you want to use other callback, change the callback parameter
    client = IRCClient(CHANNEL, NICK, PASS, init_mongodb, log_to_mongodb, close_mongodb, loop)
    coroutine = loop.create_connection(lambda: client, host, port)

    try:
        loop.run_until_complete(coroutine) # connect
        # bot send chatting
        client.send_chat('hello, I am a bot!')
        loop.run_forever() # data receiving
    except KeyboardInterrupt as e:
        print(e)
    finally:
        loop.close()
    
if __name__ == '__main__':
    main()